### 回顾
- 反爬机制
    - robots协议
    - UA检测
        - UA：User-Agent是用来伪装成浏览器的
    - 动态加载数据
- 如何检测爬取的数据是动态加载的数据？
    - 基于抓包工具进行局部搜索
- 如何捕获动态加载的数据
    - 基于抓包工具进行全局搜索
- 如何检测网站数据更新的情况
    - 增量式爬虫： 检测网站动态变化的数据
    

### 图片懒加载
- 分析：
    - 图片数据是否为动态加载的数据
        - 除了在response中进行局部搜索外，可以通过preview中查看可视化内容，判断内容是否加载出来
        - 只显示图片名称，没有显示图片
- 反爬机制：
    - 图片懒加载，只有图片显示在可视化范围之内，则图片才会被加载出来
        - 伪属性：src2，阻止图片加载，只有伪属性变成真正的src属性值图片才会被加载出来

- 面试题：爬取华为门店的数据
- 捕获到一家就可以捕获所有
- 分析，捕获的数据是否是动态加载的数据
    - 数据是动态加载的
        - 则全局搜索找到动态加载的数据的url的地址
            - url
            - 参数
        - shopid对应的一个门店的唯一标识
        - 需要获取到全部门店的id
        
- json_data_ids = requests.post(url=url, headers=headers, **json=data**).json() 或者将data反序列化 **data=json.dumps(data)**  !!!!

In [2]:
import requests
import re
url = 'http://sc.chinaz.com/tupian/fengjingtupian.html'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
page_text = requests.get(url=url, headers=headers).text  # 获取到字符串形式的响应数据
# 利用正则进行解析

ex = '<a.*?<img src2="(.*?)" alt.*?</a>'
re.findall(ex, page_text, re.S)  # re.S处理回车

['http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic26263_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26254_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26209_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26237_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26148_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26112_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26023_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic26035_s.jpg',
 'http://pic.sc.chinaz.com/Files/pic/pic9/202006/apic25993_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25843_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25813_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25795_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25789_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25723_s.jpg',
 'http://pic2.sc.chinaz.com/Files/pic/pic9/202006/apic25

### 数据解析
- 正则
- bs4
- xpath
- pyquery

### bs4解析
- 为什么需要用到bs4
    - 可以解析局部数据，实现聚焦爬虫
- 数据解析的通用原理是什么？
    - html是用来展示数据
    - 原理流程：
        - 1 标签定位
        - 2 数据的提取
- bs4数据解析的解析原理
    - 实例化一个BeautifulSoup的对象，将需要解析的数据加载到该对象中
        - 方式1：
            - BeautifulSoup(fp,'lxml'):解析本地存储的html文件
        - 方式2：
            - BeautifulSoup(page_text, 'lxml):解析网上请求到的页面数据
    - 调用BeautifulSoup对象中的相关方法和属性进行标签定位和数据的提取
- 环境的安装：
    - pip install bs4
    - pip install lxml

- 标签定义
   - soup.tagName： 返回第一次出现的tagName标签
   - 属性定位：soup.find('tagName',attrName='value') class_ 要这么写 例：soup.find('div',class_='aaa')
   - findAll和find用法是一样的，只是findAll返回的是一个列表，定位到多个标签
   - 选择器定位：定位到多个标签
       - select('selector') 例：soup.select('.happy > ul > li > a')  # > 表示一个层级 空格表示多个层级
       

- 数据的提取
    - 提取标签中存在的数据
        - soup.p.string :取出标签直系的文本内容
        - soup.p.text ：取出标签中所有的文本内容
    - 提取标签属性中存储的数据
        - tagName['attrName']

In [3]:
from bs4 import BeautifulSoup

In [5]:
# bs4中的方法和属性
fp = open('./海航集团协同办公平台.html', 'r', encoding='utf-8')
soup = BeautifulSoup(fp, 'lxml')
soup  # 对象被解析的数据

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=7" http-equiv="X-UA-Compatible"/>
<meta content="" name="keywords"/>
<meta content="" name="description"/>
<title>海航集团协同办公平台</title>
<!--[if IE 6]>
<script src="../script/iepng.js" type="text/javascript"></script>
<![endif]-->
<link href="style/basic.css" rel="stylesheet" type="text/css"/>
<!--[if IE 6]>
<script src="../script/iepng.js" type="text/javascript"></script>
<![endif]-->
<script src="../script/yu.js" type="text/javascript"></script>
<script src="../script/tb.js" type="text/javascript"></script>
<script src="../jscss/common.js" type="text/javascript"></script>
<script language="JavaScript">
//
var v_allowTopWindow = false;

/* 测试是否允许弹出窗口 */
function f_testTopWindow(){
	var testWin = window.open('','testWin','height=1px,w

In [6]:
# 标签定位
soup.div

<div id="container">
<!-- 标记(logo)区域 -->
<div id="branding">
<img src="images/head_bg.jpg"/></div>
<!-- end 标记(logo)区域 -->
<!-- 主要内容 -->
<div id="content">
<!-- 两栏结构-左为辅助内容-右为主要内容 -->
<div class="grid_c2_s4c20 box">
<div class="col_main">
<div class="main_wrap">
<!-- 右侧主要内容 -->
<!-- 广告图片 -->
<iframe frameborder="no" height="534" id="ifrdefault" scrolling="no" src="http://vpn.hnagroup.net/wifiwebui/PC/app.aspx" style="margin-top:-18px;" width="667"></iframe>
<!-- end app应用 -->
<!-- end 右侧主要内容 -->
</div>
</div>
<div class="col_sub">
<!-- 左侧边栏 -->
<!-- 登录模块 -->
<form action="../PC/logout.jsp" method="post" name="formInLogin" target="_parent">
<div class="login_success">
<div class="btn_referrals">
<input name="language" type="hidden" value="null"/>
<input name="userip" type="hidden" value="null"/>
<input name="serialNo" type="hidden" value="null"/>
<input name="basip" type="hidden" value=""/>
<input name="language" type="hidden" value="Log Out"/>
<img onclick="f_login()" src="images/btn_r

In [8]:
# 属性定位：根据属性定位的表情
soup.find('div', id='site_info')

<div id="site_info">
<p>
                版权所有<em>©</em>海航集团</p>
</div>

In [9]:
soup.select('#site_info')  # 根据id选择器定位a标签

[<div id="site_info">
 <p>
                 版权所有<em>©</em>海航集团</p>
 </div>]

In [10]:
soup.select('.border_bg')  # 根据class选择器定位a标签

[<div class="border_bg">
 <div class="linian">
 </div>
 </div>]

In [12]:
# 层级选择器
soup.select('.login_success > div > input')  # > 表示一个层级
soup.select('.login_success input')  # 空格表示多个层级

[<input name="language" type="hidden" value="null"/>,
 <input name="userip" type="hidden" value="null"/>,
 <input name="serialNo" type="hidden" value="null"/>,
 <input name="basip" type="hidden" value=""/>,
 <input name="language" type="hidden" value="Log Out"/>]

In [18]:
soup.p.text

'\n                版权所有©海航集团'

In [20]:
soup.img['src']

'images/head_bg.jpg'

In [28]:
# bs4爬取三国演义小说数据
url = 'http://www.shicimingju.com/book/sanguoyanyi.html'
page_text = requests.get(url=url, headers=headers).text
fp = open('./sanguo.txt', 'w', encoding='utf-8')
soup = BeautifulSoup(page_text, 'lxml')
a_list = soup.select('.book-mulu > ul > li > a')
for a in a_list:
    title = a.string
    detail_url = 'http://www.shicimingju.com'+a['href']

    # 解析提取章节内容
    detail_page = requests.get(url=detail_url, headers=headers).text
    # 解析详情页中的章节内容
    d_soup = BeautifulSoup(detail_page, 'lxml')
    content = d_soup.find('div', class_='chapter_content').text
    fp.write(title+':' + content+'\n')
    print(title, '下载完成')
fp.close()

### xpath数据解析原理
- html标签结构
    - 是一个树状结构
- xpath解析原理
    - 实例化一个etree对象，且将即将被解析的数据加载到对象中
        - 解析本地存储的html文档
            - etree.parse('filename')
            - parser = etree.HTMLParser(encoding='utf-8') # html不规范时手动添加parser参数
        - 解析互联网爬取的html数据
            - etree.HTML(page_text)
            
    - 使用etree对象中xpath方法结合着不同的xpath表达式实现标签和定位，重点是不同的xpath表达式
    

### xpath表达式
- 标签定位
    - 最左侧的/:必须要从根标签开始逐层的定位目标标签
    - 不是最左侧的/:表示一个层级
    - 最左侧的//:可以从任意位置定义目标标签
    - 不是最左侧的//:表示多个层级
    
    - 属性定位://tagName[@attrName="sss"]
    - 索引定位://tagName[index],index的索引是从1开始的
    - 模糊匹配：
        - //div[contains(@class, 'ng')] 定位到class属性包含ng的div标签
        - //div[starts-with(@class, 'ta')] 定位到class属性值中以ta开头的div标签
- 数据解析
    - 取标签中的数据
        - /text() :直系文本内容，相当于bs4中的string
        - //text()：标签中的所有的内容，相当于bs4中的text
    - 取属性的数据
        - tagName/@attrName
    - 提升数据解析的通用性
        - 两个表达式通过管道符连接 |

In [31]:
from lxml import etree

In [45]:
parser = etree.HTMLParser(encoding='utf-8')  # html不规范时手动添加parser参数
tree = etree.parse('./test.html', parser=parser)

In [46]:
tree

In [47]:
tree.xpath('/html/head')  # 从根标签开始定位head标签
tree.xpath('//head')  # 将标签中所有的head标签定位到

[<Element head at 0x2602eb36c88>]

In [50]:
tree.xpath('//div[@class="tt"]//text()')

['相关搜索']

In [59]:
import os

In [60]:
# 爬取图片数据和图片名称将其保存到本地
# 第一页：http://pic.netbian.com/4kdongman/
# 不是第一页：http://pic.netbian.com/4kdongman/index_2.html
dirname ='./imglibs' 
if not os.path.exists(dirname):
    os.mkdir(dirname)
url = 'http://pic.netbian.com/4kdongman/index_{}.html'
for page in range(1, 6):
    if page == 1:
        new_url = url
    new_url = url.format(page)
    res = requests.get(new_url, headers=headers)
    res.encoding = 'gbk'
    page_text = res.text
    # 数据解析：图片地址和图片名称
    tree = etree.HTML(page_text)
    li_list = tree.xpath('//div[@class="slist"]/ul/li')  # 全局数据解析
    for li in li_list:
        title = li.xpath('./a/img/@alt')[0]+'.jpg' # 局部数据解析
        img_url = 'http://pic.netbian.com/'+li.xpath('./a/img/@src')[0]
        img_data = requests.get(url=img_url, headers=headers).content
        file_path = dirname + '/'+title
        with open(file_path, 'wb') as f:
            f.write(img_data)
            print(title, '下载完成')

Epic7第七史诗动漫美女4k高清壁纸.jpg 下载完成
鬼灭之刃蝴蝶忍4k高清电脑壁纸3840x2160.jpg 下载完成
女孩子 校服 书包 鸟儿 回家的路上 4k动漫壁纸.jpg 下载完成
女孩 紫色短发 连衣裙 白色裤袜 坐在凉席上 茶 凉鞋 樱花 4k动漫壁纸.jpg 下载完成
女孩 侧坐在木地板上 室内 窗户 阳光 唯美艺术4k动漫人物壁纸.jpg 下载完成
森林 花草树木 蝴蝶 刀剑神域 亚斯娜4k高清动漫壁纸.jpg 下载完成
女孩 微笑 蹲着 水 海岸 海滩 波浪 风 夜晚 4k动漫壁纸.jpg 下载完成
女孩喝啤酒 飘窗 城市夜景 雨天 猫 好看唯美4k动漫壁纸.jpg 下载完成
动漫女孩 高挑美女 坐着 郎腿 小鸟 猫 唯美艺术动漫4k壁纸.jpg 下载完成
dva 宅女 长筒袜 短发 可爱女孩4k壁纸.jpg 下载完成
雨天透过车窗的女孩4k动漫壁纸3840x2160.jpg 下载完成
可爱女生 单车 自行车 运动女孩 4k动漫壁纸.jpg 下载完成
Aerith艾瑞丝 女孩子 花 唯美插图4k高清动漫壁纸.jpg 下载完成
女孩子 鯨鱼 4k高清动漫壁纸.jpg 下载完成
Fate Saber 阿尔托利亚潘德拉贡4k高清动漫壁纸.jpg 下载完成
女仆装 黄色头发 可爱动漫美女4k壁纸3840x2160.jpg 下载完成
Aerith艾瑞丝4k高清电脑壁纸.jpg 下载完成
《伊什塔尔Ishtar》4k高清壁纸.jpg 下载完成
美女 天使 彩色头发 梦幻唯美4k壁纸3840x2160.jpg 下载完成
少女 女孩 耳朵 兽耳 长辫子背部 唯美4k动漫壁纸.jpg 下载完成
公主 长卷发 美丽的裙子 蝴蝶 唯美4k动漫壁纸.jpg 下载完成
白银圭 水手服 白色连衣裙 女子 高中生 4k动漫壁纸.jpg 下载完成
女子 种子 花 4k动漫壁纸3840x2160.jpg 下载完成
雪 女孩唯美插画4k高清动漫壁纸.jpg 下载完成
美女剑客小姐姐二次元高清动漫壁纸.jpg 下载完成
女生 女子 起床 晚上 都市 夜景 4k动漫壁纸.jpg 下载完成
蓝色海水 水下女孩 海龟 鱼 唯美意境4k动漫壁纸.jpg 下载完成
总有一天 女孩 城市 日落 天空 云 道路 唯美意境4k动漫壁纸3840x2160.jpg 下载完成
穿白色裙子女孩  台阶 

In [65]:
# 提升xpath表达式的通用性
url = 'https://www.aqistudy.cn/historydata/'
page_text = requests.get(url=url, headers=headers).text
tree = etree.HTML(page_text)
all_name = tree.xpath('//div[@class="bottom"]/ul/li/a/text() | //div[@class="bottom"]/ul/div[2]/li/a/text()') # 利用管道符提升xpath表达式的通用性
all_name

['北京',
 '上海',
 '广州',
 '深圳',
 '杭州',
 '天津',
 '成都',
 '南京',
 '西安',
 '武汉',
 '阿坝州',
 '安康',
 '阿克苏地区',
 '阿里地区',
 '阿拉善盟',
 '阿勒泰地区',
 '安庆',
 '安顺',
 '鞍山',
 '克孜勒苏州',
 '安阳',
 '蚌埠',
 '白城',
 '保定',
 '北海',
 '宝鸡',
 '北京',
 '毕节',
 '博州',
 '百色',
 '白沙',
 '白山',
 '保山',
 '保亭',
 '包头',
 '本溪',
 '白银',
 '巴彦淖尔',
 '滨州',
 '巴中',
 '亳州',
 '长春',
 '承德',
 '成都',
 '常德',
 '昌都',
 '赤峰',
 '昌江',
 '昌吉州',
 '五家渠',
 '澄迈',
 '重庆',
 '常熟',
 '长沙',
 '楚雄州',
 '朝阳',
 '滁州',
 '郴州',
 '潮州',
 '常州',
 '长治',
 '崇左',
 '沧州',
 '池州',
 '定安',
 '丹东',
 '东方',
 '东莞',
 '德宏州',
 '大连',
 '大理州',
 '大庆',
 '大同',
 '定西',
 '大兴安岭地区',
 '黔南州',
 '德阳',
 '东营',
 '达州',
 '德州',
 '儋州',
 '鄂尔多斯',
 '恩施州',
 '鄂州',
 '防城港',
 '抚顺',
 '佛山',
 '阜新',
 '阜阳',
 '富阳',
 '福州',
 '抚州',
 '广安',
 '贵港',
 '果洛州',
 '桂林',
 '甘南州',
 '贵阳',
 '广元',
 '固原',
 '广州',
 '甘孜州',
 '赣州',
 '淮安',
 '淮北',
 '鹤壁',
 '海北州',
 '河池',
 '邯郸',
 '海东地区',
 '哈尔滨',
 '合肥',
 '黄冈',
 '鹤岗',
 '红河州',
 '怀化',
 '黑河',
 '呼和浩特',
 '海口',
 '呼伦贝尔',
 '葫芦岛',
 '海门',
 '哈密地区',
 '淮南',
 '黄南州',
 '海南州',
 '黄山',
 '衡水',
 '黄石',
 '和田地区',
 '海西州',
 '衡阳',
 '河源',
 '湖州',
 '汉中',
 '杭州'

- 面试题：
    - 想要解析出一张页面中的指定的局部带标签的数据。
        - 使用bs4解析定位标签后，直接返回的就是标签的内容

In [66]:
soup.a

<a href="/"><img id="logo" src="/public/image/logo.png"/></a>